In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
# plt.style.use('seaborn-paper')
import seaborn as sns
import joblib
# plt.rcParams['font.family']='Times New Roman,Microsoft YaHei'# 设置字体族，中文为微软雅黑，英文为Times New Roman
plt.rcParams['font.sans-serif'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'  # 设置数学公式字体为stix
plt.rcParams["text.usetex"] = False
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from sklearn.metrics import mean_squared_error

In [ ]:
datajh=pd.read_csv("groupedjhS.csv")
datacm=pd.read_csv("groupedcmS.csv")
datajhsoa=pd.read_csv("groupedjhSOA.csv")
datacmsoa=pd.read_csv("groupedcmSOA.csv")

In [ ]:
# datajh=pd.read_csv("groupedjhall.csv")
# datacm=pd.read_csv("groupedcmall.csv")
# datajhsoa=datajh
# datacmsoa=datacm
# 
# datajhsoa["SOA"] = 0
# datajh["SOA"] = 0
# for i in datajhsoa.columns[(datajhsoa.columns.get_loc("0.25um")):(datajhsoa.columns.get_loc("0.28um"))]:
#     datajhsoa["SOA"] = datajhsoa["SOA"] + datajhsoa[i]
#     datajh["SOA"] = datajh["SOA"] + datajh[i]
# datacmsoa["SOA"] = 0
# datacm["SOA"] = 0
# for i in datacmsoa.columns[(datacmsoa.columns.get_loc("0.25um")):(datacmsoa.columns.get_loc("0.28um"))]:
#     datacmsoa["SOA"] = datacmsoa["SOA"] + datacmsoa[i]
#     datacm["SOA"] = datacm["SOA"] + datacm[i]

In [ ]:
VOCs = ['Methyl Mercaptan', '1,3-Butadiene', 'Butene', 'Acetone/Butane', 'n-Propanol', 'Dimethyl Sulfide/Ethyl Mercaptan', 'Chloroethane', 'Isoprene', 'Pentene', 'Pentane/Isopentane', 'Dimethylformamide', 'Ethyl Formate', 'Carbon Disulfide/Propyl Mercaptan', 'Benzene', 'Cyclohexene', 'Hexene/Methylcyclopentane', 'n-Hexane/Dimethylbutane', 'Ethyl Sulfide/Butyl Mercaptan', 'Toluene', 'Aniline', 'Dimethyl Disulfide', '1,1-Dichloroethylene', 'Methylcyclohexane', 'n-Heptane', 'Triethylamine', 'n-Propyl Acetate', 'Diethylene Triamine', 'Styrene', 'Xylene/Ethylbenzene', '1,3-Dichloropropene', 'n-Octane', 'n-Butyl Acetate', 'Hexyl Mercaptan', 'Xylenol', 'Trichloroethylene', 'Diethylbenzene', 'Methyl Benzoate', 'Trimethyl Phosphate', 'n-Decanol', 'Dichlorobenzene', 'Diethyl Aniline', 'Undecane', 'Tetrachloroethylene', 'n-Dodecane', 'Dibromomethane', '1,2,4-Trichlorobenzene', 'n-Tridecane', '1,2-Dibromoethane']

In [ ]:

datajhsoa["SOA"] = 0
datajh["SOA"] = 0
for i in datajhsoa.columns[(datajhsoa.columns.get_loc("0.25um")):(datajhsoa.columns.get_loc("0.28um"))]:
    datajhsoa["SOA"] = datajhsoa["SOA"] + datajhsoa[i]
    datajh["SOA"] = datajh["SOA"] + datajh[i]
datajhsoa["CSOA"] = 0
for i in datajhsoa.columns[(datajhsoa.columns.get_loc("C0.25um")):(datajhsoa.columns.get_loc("C0.28um"))]:
    datajhsoa["CSOA"] = datajhsoa["CSOA"] + datajhsoa[i]
datacmsoa["SOA"] = 0
datacm["SOA"] = 0
for i in datacmsoa.columns[(datacmsoa.columns.get_loc("0.25um")):(datacmsoa.columns.get_loc("0.28um"))]:
    datacmsoa["SOA"] = datacmsoa["SOA"] + datacmsoa[i]
    datacm["SOA"] = datacm["SOA"] + datacm[i]
datacmsoa["CSOA"] = 0
for i in datacmsoa.columns[(datacmsoa.columns.get_loc("C0.25um")):(datacmsoa.columns.get_loc("C0.28um"))]:
    datacmsoa["CSOA"] = datacmsoa["CSOA"] + datacmsoa[i]

In [ ]:
datajh['place']='JH'
datacm['place']='CM'
dataall=pd.concat([datajh,datacm],axis=0)
dataall.columns = ['Time', 'TVOCs', 'Methyl Mercaptan', '1,3-Butadiene', 'Butene', 'Acetone/Butane', 'n-Propanol',
                   'Dimethyl Sulfide/Ethyl Mercaptan', 'Chloroethane', 'Isoprene', 'Pentene', 'Pentane/Isopentane',
                   'Dimethylformamide', 'Ethyl Formate', 'Carbon Disulfide/Propyl Mercaptan', 'Benzene', 'Cyclohexene',
                   'Hexene/Methylcyclopentane', 'n-Hexane/Dimethylbutane', 'Ethyl Sulfide/Butyl Mercaptan', 'Toluene',
                   'Aniline', 'Dimethyl Disulfide', '1,1-Dichloroethylene', 'Methylcyclohexane', 'n-Heptane',
                   'Triethylamine', 'n-Propyl Acetate', 'Diethylene Triamine', 'Styrene', 'Xylene/Ethylbenzene',
                   '1,3-Dichloropropene', 'n-Octane', 'n-Butyl Acetate', 'Hexyl Mercaptan', 'Xylenol',
                   'Trichloroethylene', 'Diethylbenzene', 'Methyl Benzoate', 'Trimethyl Phosphate', 'n-Decanol',
                   'Dichlorobenzene', 'Diethyl Aniline', 'Undecane', 'Tetrachloroethylene', 'n-Dodecane',
                   'Dibromomethane', '1,2,4-Trichlorobenzene', 'n-Tridecane', '1,2-Dibromoethane', '0.25um', '0.28um',
                   '0.30um', '0.35um', '0.40um', '0.45um', '0.50um', '0.58um', '0.65um', '0.70um', '0.80um', '1.00um',
                   '1.30um', '1.60um', '2.00um', '2.50um', '3.00um', '3.50um', '4.00um', '5.00um', '6.50um', '7.50um',
                   '8.50um', '10.00um', '12.50um', '15.00um', '17.50um', '20.00um', '25.00um', '30.00um', '32.00um',
                   'PM10', 'PM2.5', 'PM1', 'SO2', 'NOx', 'NO', 'NO2', 'CO', 'O3', 'NO2.1', 'NegativeOxygenIons',
                   'Radiation', 'Temperature', 'Humidity', 'WindSpeed', 'Hour_Min_Sec', 'Hour_Min', 'Hour', 'Month',
                   'Day', 'Datetime', 'seconds', 'SOA', 'place']
dataall

In [ ]:
from scipy.stats import norm, expon, gamma, lognorm, beta, kstest, shapiro
def normal_distribution_fit_and_test(grouped_by_hour):
    normality_results = {}
    for hour, group in grouped_by_hour:
        # Fit normal distribution and get parameters
        params = norm.fit(group)
        # Perform Shapiro-Wilk test
        _, p_value = shapiro(group)
        # Check if data is normally distributed based on p-value
        is_normal = True if p_value > 0.05 else False
        normality_results[hour] = (
        params[0], params[1], p_value, is_normal)  # params[0]: mean, params[1]: standard deviation

    normality_df = pd.DataFrame(normality_results).T
    normality_df.columns = ["Mean", "Standard Deviation", "P-Value", "Is Normal"]
    normality_df.reset_index(inplace=True)
    normality_df.rename(columns={'index': 'Hour'}, inplace=True)

    return normality_df



In [ ]:
from scipy.optimize import curve_fit


# 定义与均值 Mean 的关系的函数
def mean_relation(T, Q0, a, v0):
    return Q0 + (a * T ** 2) / 2 + T * v0


# 定义与方差 variance 的关系的函数
def std_dev_relation(T, k, sigma0):
    return ((k ** 2) * (T ** 3)) / 3 + k * (T ** 2) * sigma0 + T * (sigma0 ** 2)


from scipy.stats import zscore
from scipy.stats import t


# 定义去除离群值的函数（使用 Z-score 方法）
def remove_outliers(data):
    z_scores = zscore(data)
    return data[(np.abs(z_scores) < 3)]  # 通常使用 3 作为 Z-score 的阈值


# 定义加权二次关系模型
def weighted_quadratic_relation(T, y, weights):
    # 增加常数项和二次项
    T_with_const = sm.add_constant(np.column_stack((T, T**2)))
    # 加权最小二乘法（WLS）
    model_wls = sm.WLS(y, T_with_const, weights=weights).fit()
    return model_wls.rsquared  # 返回加权的 R² 值

def clean_data(df, columns, threshold=3):
    for col in columns:
        df = df[np.abs(zscore(df[col])) < threshold]
    return df.reset_index(drop=True)


In [ ]:
# from scipy.spatial import ConvexHull
# from shapely.geometry import Polygon
# from shapely.ops import unary_union
# from scipy.stats import norm



# def compute_area(points):
#     # 计算点的凸包区域
#     hull = ConvexHull(points)
#     polygon = Polygon(points[hull.vertices])
#     return polygon.area
# 
# 
# # 蒙特卡罗模拟方法计算重叠面积和综合拟合度
# def monte_carlo_r_squared_area(T, mean_params, std_dev_params, real_data, num_simulations=1000, w=0, penalty_coefficient=1.0):
#     original_points = np.column_stack((T, real_data))
#     original_area = compute_area(original_points)
# 
#     simulated_means = mean_relation(T, *mean_params)
#     residuals = real_data - simulated_means
#     ss_res = np.sum(residuals ** 2)
#     ss_tot = np.sum((real_data - np.mean(real_data)) ** 2)
#     r_squared_mean = 1 - (ss_res / ss_tot)
# 
#     real_var = np.var(real_data)
#     simulation_areas = []
#     overlap_areas = []
#     penalty_total = 0
# 
#     for _ in range(num_simulations):
#         simulated_path = norm.rvs(
#             loc=mean_relation(T, *mean_params),
#             scale=np.sqrt(std_dev_relation(T, *std_dev_params)),
#             size=len(T)
#         )
#         simulated_points = np.column_stack((T, simulated_path))
#         sim_area = compute_area(simulated_points)
#         simulation_areas.append(sim_area)
# 
#         # 计算模拟路径的方差，并计算惩罚项
#         sim_var = np.var(simulated_path)
#         penalty = max(0, (sim_var - real_var) / real_var)  # 方差相对偏差
#         penalty_total += penalty
# 
#         original_polygon = Polygon(original_points[ConvexHull(original_points).vertices])
#         simulated_polygon = Polygon(simulated_points[ConvexHull(simulated_points).vertices])
#         intersection_area = original_polygon.intersection(simulated_polygon).area
#         overlap_areas.append(intersection_area)
# 
#     mean_overlap_area = np.mean(overlap_areas)
# 
#     # 调整后的 R² 计算
#     r_squared_adjusted_area = mean_overlap_area / (original_area + penalty_coefficient * penalty_total)
#     r_squared_adjusted_area = min(max(r_squared_adjusted_area, 0), 1)  # 限制 R² 在 [0, 1]
# 
#     # 综合 R²
#     r_squared_combined = w * r_squared_mean + (1 - w) * r_squared_adjusted_area
#     return r_squared_combined

# def monte_carlo_r_squared_area(T, mean_params, std_dev_params, real_data, num_simulations=1000):
#     original_points = np.column_stack((T, real_data))
#     original_area = compute_area(original_points)
# 
#     simulation_areas = []
#     overlap_areas = []
# 
#     for _ in range(num_simulations):
#         simulated_path = norm.rvs(
#             loc=mean_relation(T, *mean_params),
#             scale=np.sqrt(std_dev_relation(T, *std_dev_params)),
#             size=len(T)
#         )
#         simulated_points = np.column_stack((T, simulated_path))
#         sim_area = compute_area(simulated_points)
#         simulation_areas.append(sim_area)
# 
#         original_polygon = Polygon(original_points[ConvexHull(original_points).vertices])
#         simulated_polygon = Polygon(simulated_points[ConvexHull(simulated_points).vertices])
#         intersection_area = original_polygon.intersection(simulated_polygon).area
#         overlap_areas.append(intersection_area)
# 
#     mean_overlap_area = np.mean(overlap_areas)
#     mean_simulation_area = np.mean(simulation_areas)
# 
#     # Calculate R² based on area overlap
#     r_squared_area = mean_overlap_area / original_area
#     return min(max(r_squared_area, 0), 1)  # Ensure R² is in [0, 1]


In [ ]:
# from scipy.spatial import ConvexHull
# from shapely.geometry import Polygon
# from shapely.ops import unary_union
# from scipy.stats import norm
# 
# def compute_area(points):
#     """
#     计算点集的凸包区域面积。
#     """
#     if len(points) < 3:
#         return 0
#     hull = ConvexHull(points)
#     polygon = Polygon(points[hull.vertices])
#     return polygon.area
# 
# def monte_carlo_r_squared_area(
#     T,
#     mean_params,
#     std_dev_params,
#     real_data,
#     num_simulations=1000,
#     penalty_coefficient=1.0
# ):
#     """
#     蒙特卡罗模拟方法计算调整后的 R²，考虑新的方差惩罚项。
#     T: 自变量（例如温度）
#     mean_params: 拟合的均值函数参数
#     std_dev_params: 拟合的方差函数参数
#     real_data: 原始观测数据
#     num_simulations: 模拟次数
#     penalty_coefficient: 惩罚系数，用于调整方差惩罚的影响程度
#     """
#     # 原始数据点
#     original_points = np.column_stack((T, real_data))
#     original_area = compute_area(original_points)
# 
#     # 原始数据的方差
#     real_var = np.var(real_data)
# 
#     overlap_areas = []
#     penalties = []
# 
#     for _ in range(num_simulations):
#         # 根据均值和方差函数生成模拟数据
#         simulated_path = norm.rvs(
#             loc=mean_relation(T, *mean_params),
#             scale=np.sqrt(std_dev_relation(T, *std_dev_params)),
#             size=len(T)
#         )
#         simulated_points = np.column_stack((T, simulated_path))
# 
#         # 计算模拟数据与原始数据的重叠面积
#         original_polygon = Polygon(original_points[ConvexHull(original_points).vertices])
#         simulated_polygon = Polygon(simulated_points[ConvexHull(simulated_points).vertices])
#         intersection = original_polygon.intersection(simulated_polygon)
#         intersection_area = intersection.area if not intersection.is_empty else 0
#         overlap_areas.append(intersection_area)
# 
#         # 计算模拟数据的方差，计算惩罚项
#         sim_var = np.var(simulated_path)
#         if sim_var > real_var:
#             penalty = ((sim_var / real_var) - 1) ** 2  # 方差比的平方惩罚
#         else:
#             penalty = 0  # 如果模拟方差不超过真实方差，则无惩罚
#         penalties.append(penalty)
# 
#     # 计算平均重叠面积和平均惩罚
#     mean_overlap_area = np.mean(overlap_areas)
#     mean_penalty = np.mean(penalties)
# 
#     # 计算调整后的 R²，考虑新的方差惩罚项
#     adjusted_r_squared = mean_overlap_area / (original_area * (1 + penalty_coefficient * mean_penalty))
#     adjusted_r_squared = np.clip(adjusted_r_squared, 0, 1)  # 限制 R² 在 [0, 1] 之间
# 
#     return adjusted_r_squared

In [ ]:
# from scipy.stats import gaussian_kde, norm
# 
# 
# def kl_divergence(p, q, x_vals):
#     """
#     计算两个概率密度之间的KL散度
#     """
#     return np.sum(p * np.log(p / (q + 1e-10))) * (x_vals[1] - x_vals[0])
# 
# def monte_carlo_density_r_squared(
#     T,
#     mean_params,
#     std_dev_params,
#     real_data,
#     num_simulations=1000,
#     penalty_coefficient=1
# ):
#     """
#     基于概率密度的蒙特卡罗模拟方法计算调整后的 R²，考虑方差惩罚。
#     T: 自变量（例如温度）
#     mean_params: 拟合的均值函数参数
#     std_dev_params: 拟合的方差函数参数
#     real_data: 原始观测数据
#     num_simulations: 模拟次数
#     penalty_coefficient: 惩罚系数，用于调整方差惩罚的影响程度
#     """
#     # 计算观测数据的概率密度
#     density_obs = gaussian_kde(real_data)
#     x_vals = np.linspace(min(real_data), max(real_data), 100)
#     p_obs = density_obs(x_vals)
#     
#     # 计算空模型（正态分布）的概率密度
#     mean_obs, std_obs = np.mean(real_data), np.std(real_data)
#     p_null = norm.pdf(x_vals, loc=mean_obs, scale=std_obs)
#     kl_null = kl_divergence(p_obs, p_null, x_vals)
# 
#     # 模拟路径的KL散度和方差惩罚
#     kl_total = 0
#     penalties = []
# 
#     for _ in range(num_simulations):
#         # 生成模拟路径
#         simulated_path = norm.rvs(
#             loc=mean_relation(T, *mean_params),
#             scale=np.sqrt(std_dev_relation(T, *std_dev_params)),
#             size=len(T)
#         )
#         
#         # 计算模拟路径的概率密度
#         density_sim = gaussian_kde(simulated_path)
#         p_sim = density_sim(x_vals)
#         
#         # 计算观测数据与模拟数据的KL散度
#         kl_sim = kl_divergence(p_obs, p_sim, x_vals)
#         kl_total += kl_sim
#         
#         # 计算模拟路径的方差，并计算方差惩罚
#         sim_var = np.var(simulated_path)
#         real_var = np.var(real_data)
#         if sim_var > real_var:
#             penalty = ((sim_var / real_var) - 1) ** 2  # 方差比的平方惩罚
#         else:
#             penalty = 0  # 如果模拟方差不超过真实方差，则无惩罚
#         penalties.append(penalty)
# 
#     # 计算平均KL散度和平均方差惩罚
#     kl_avg = kl_total / num_simulations
#     mean_penalty = np.mean(penalties)
# 
#     # 计算调整后的 R²，考虑KL散度和方差惩罚
#     adjusted_r_squared = 1 - (kl_avg / kl_null) / (1 + penalty_coefficient * mean_penalty)
#     adjusted_r_squared = np.clip(adjusted_r_squared, 0, 1)  # 限制 R² 在 [0, 1] 之间
# 
#     return adjusted_r_squared


In [1]:
from scipy.stats import (
    gaussian_kde,
    norm,
    uniform,
    gamma,
    weibull_min,
    pearson3  # 用于对数皮尔逊III型分布
)

def kl_divergence(p, q, x_vals):
    """
    计算两个概率密度之间的KL散度
    """
    return np.sum(p * np.log(p / (q + 1e-10))) * (x_vals[1] - x_vals[0])

def adaptive_penalty_coefficient(sim_var, real_var):
    variance_ratio = sim_var / real_var
    if variance_ratio > 1:
        return np.exp(variance_ratio)
    else:
        return np.log(variance_ratio+1) / np.log(100)  # 如果模拟方差不超过真实方差，则轻微惩罚
        # return 0  

def monte_carlo_density_r_squared(
    T,
    mean_params,
    std_dev_params,
    real_data,
    num_simulations=1000,
    distribution='uniform'  # 新增参数，默认为 'uniform'
):
    """
    基于概率密度的蒙特卡罗模拟方法计算调整后的 R²，考虑自适应的方差惩罚。

    T: 自变量（例如温度）
    mean_params: 拟合的均值函数参数
    std_dev_params: 拟合的方差函数参数
    real_data: 原始观测数据
    num_simulations: 模拟次数
    distribution: 零模型的分布类型，支持 'normal', 'uniform', 'gamma', 'weibull', 'logpearson3'
    """
    # 计算观测数据的概率密度
    density_obs = gaussian_kde(real_data)
    x_vals = np.linspace(np.min(real_data), np.max(real_data), 100)
    p_obs = density_obs(x_vals)
    
    simv=[]
    ov=[]

    # 根据指定的分布类型计算零模型的概率密度
    if distribution == 'normal':
        mean_obs, std_obs = np.mean(real_data), np.std(real_data)
        p_null = norm.pdf(x_vals, loc=mean_obs, scale=std_obs)
    elif distribution == 'uniform':
        a, b = np.min(real_data), np.max(real_data)
        p_null = uniform.pdf(x_vals, loc=a, scale=b - a)
    elif distribution == 'gamma':
        params = gamma.fit(real_data, floc=0)  # 固定 loc=0，避免负值
        shape, loc, scale = params
        p_null = gamma.pdf(x_vals, a=shape, loc=loc, scale=scale)
    elif distribution == 'weibull':
        params = weibull_min.fit(real_data, floc=0)  # 固定 loc=0
        c, loc, scale = params
        p_null = weibull_min.pdf(x_vals, c=c, loc=loc, scale=scale)
    elif distribution == 'logpearson3':
        # 对数据取对数
        positive_data = real_data[real_data > 0]  # 确保数据为正
        log_data = np.log(positive_data)
        # 拟合 Pearson III 型分布
        skew, loc, scale = pearson3.fit(log_data)
        # 计算对数空间的概率密度
        log_x_vals = np.log(x_vals[x_vals > 0])  # x_vals 也要为正
        p_null_log = pearson3.pdf(log_x_vals, skew=skew, loc=loc, scale=scale)
        # 转换回原始空间的概率密度
        p_null = np.zeros_like(x_vals)
        p_null[x_vals > 0] = (1 / x_vals[x_vals > 0]) * p_null_log
    else:
        raise ValueError(f"不支持的分布类型：{distribution}")

    kl_null = kl_divergence(p_obs, p_null, x_vals)

    # 模拟路径的KL散度和自适应惩罚
    kl_total = 0
    penalty_total = 0

    for _ in range(num_simulations):
        # 生成模拟路径
        simulated_path = np.random.normal(
            loc=mean_relation(T, *mean_params),
            scale=np.sqrt(std_dev_relation(T, *std_dev_params)),
            size=len(T)
        )
        
        # 计算模拟路径的概率密度
        density_sim = gaussian_kde(simulated_path)
        p_sim = density_sim(x_vals)
        
        # 计算观测数据与模拟数据的KL散度
        kl_sim = kl_divergence(p_obs, p_sim, x_vals)
        kl_total += kl_sim
        
        # 计算模拟路径的方差，并计算自适应惩罚
        sim_var = np.var(simulated_path)
        real_var = np.var(real_data)
        penalty = adaptive_penalty_coefficient(sim_var, real_var)
        penalty_total += penalty
        simv.append(sim_var)
        ov.append(real_var)

    # 计算平均KL散度和平均自适应惩罚
    kl_avg = kl_total / num_simulations
    mean_penalty = penalty_total / num_simulations

    
    print(np.mean(simv)/np.mean(ov))

    # 计算调整后的 R²，考虑自适应的方差惩罚
    adjusted_r_squared = (1 - (kl_avg / kl_null)) / (1 + mean_penalty)
    print(adjusted_r_squared)
    adjusted_r_squared = np.clip(adjusted_r_squared, 0, 1)  # 限制 R² 在 [0, 1] 之间

    return adjusted_r_squared

In [ ]:
# 加载已保存的 KMeans 模型
kmeans = joblib.load('kmeans.joblib')
# 获取每个化学物质的聚类标签并存储为字典
chemical_clusters = dict(zip(VOCs, kmeans.labels_))


# 定义处理化学物质数据的函数
def process_chemical(data, chemical_name):
    print(chemical_name)
    data_grouped = data.groupby(["Hour_Min"]).mean(numeric_only=True).reset_index()
    data_grouped["Concentration"] = data_grouped[chemical_name]
    grouped_by_hour = data_grouped.groupby('Hour')
    normality_df = normal_distribution_fit_and_test(grouped_by_hour[chemical_name])
    normality_df['T'] = data_grouped.groupby('Hour').mean(numeric_only=True)["Temperature"]
    
    # data_grouped = data.copy()
    # data_grouped["Concentration"] = data_grouped[chemical_name]
    # grouped_by_hour = data_grouped.groupby('Hour_Min')
    # normality_df = normal_distribution_fit_and_test(grouped_by_hour[chemical_name])
    # normality_df['T'] = data_grouped.groupby('Hour_Min').mean(numeric_only=True)["Temperature"].values

    # 提取数据
    T = normality_df['T'].values
    mean_values = normality_df['Mean'].astype("float").values
    std_dev_values = normality_df['Standard Deviation'].astype("float").values ** 2

    # 去除离群值
    mean_values_filtered = remove_outliers(mean_values)
    std_dev_values_filtered = remove_outliers(std_dev_values)
    T_filtered_mean = T[np.isin(mean_values, mean_values_filtered)]
    T_filtered_std_dev = T[np.isin(std_dev_values, std_dev_values_filtered)]
    

    # 拟合模型
    params_mean, covariance_mean = curve_fit(mean_relation, T_filtered_mean, mean_values_filtered, method='trf', maxfev=1000)
    params_std_dev, covariance_std_dev = curve_fit(std_dev_relation, T_filtered_std_dev, std_dev_values_filtered, method='trf', maxfev=1000,
    bounds=([0, -np.inf], [np.inf, np.inf]))

    # 计算统计量
    p_values_mean = calculate_p_values(params_mean, covariance_mean, len(T_filtered_mean))
    p_values_std_dev = calculate_p_values(params_std_dev, covariance_std_dev, len(T_filtered_std_dev))
    r_squared_mean = calculate_r_squared(mean_values_filtered, mean_relation, T_filtered_mean, params_mean)
    r_squared_std_dev = calculate_r_squared(std_dev_values_filtered, std_dev_relation, T_filtered_std_dev, params_std_dev)
    
    # 计算占比 %
    total_mean_concentration = data[VOCs].mean().sum()  # 总体均值
    percentage = (np.mean(mean_values_filtered) / total_mean_concentration) * 100  # 占比 %
    
    cluster_label = chemical_clusters.get(chemical_name, None)
    
    weight=std_dev_relation(T_filtered_mean, *params_std_dev)
    weights = 1 / (weight + 1e-8)
    wls_r_squared = weighted_quadratic_relation(T_filtered_mean, mean_values_filtered, weights)
    
    data_grouped = clean_data(data_grouped, ['Temperature', chemical_name])
    print(data_grouped.shape)

    
    
    # 计算伪 R²

    rtotal=  monte_carlo_density_r_squared(
        data_grouped['Temperature'].values, params_mean, params_std_dev, data_grouped[[chemical_name]].values.T
    )
    
    

    # 整理结果
    return {
        'Chemical': chemical_name,
        'Cluster': cluster_label+1,
        'Average Concentration': np.mean(mean_values_filtered),
        'R2 Total':rtotal,
        'R2 WLS':wls_r_squared,
        'R2 Mean': r_squared_mean,
        'Concentration Std Dev': np.std(mean_values_filtered),
        'R2 Std Dev': r_squared_std_dev,
        'Percentage (%)': percentage,
        'Q0': params_mean[0],
        'Pvalue Q0': p_values_mean[0],
        'a': params_mean[1],
        'Pvalue a': p_values_mean[1],
        'v0': params_mean[2],
        'Pvalue v0': p_values_mean[2],
        'k': params_std_dev[0],
        'Pvalue k': p_values_std_dev[0],
        'sigma0': params_std_dev[1],
        'Pvalue sigma0': p_values_std_dev[1]
    }

# 定义用于计算 P 值和 R² 的辅助函数
def calculate_p_values(params, covariance, n):
    std_errors = np.sqrt(np.diag(covariance))
    t_values = params / std_errors
    dof = max(0, n - len(params))
    p_values = 2 * (1 - t.cdf(np.abs(t_values), dof))
    return p_values

def calculate_r_squared(data, fit_function, T, params):
    residuals = data - fit_function(T, *params)
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((data - np.mean(data)) ** 2)
    return 1 - (ss_res / ss_tot)


In [ ]:
# 计算每种VOC的平均浓度
average_concentrations = dataall[VOCs].mean()
# 计算所有VOCs的平均总浓度
total_concentration = average_concentrations.sum()
# 计算每种VOC占总浓度的百分比
concentration_percentage = (average_concentrations / total_concentration) * 100
# 筛选出占比大于2%的VOCs
selected_chemicals = concentration_percentage[concentration_percentage > 2].index.tolist()

In [ ]:
chemicals = selected_chemicals

In [ ]:
# 处理 JH 地点的化学物质数据
results_jh = [process_chemical(dataall[dataall.place == 'JH'], chemical) for chemical in chemicals]
results_df_jh = pd.DataFrame(results_jh)
results_df_jh['Place'] = 'JH'  # 添加地点信息

# 处理 CM 地点的化学物质数据
results_cm = [process_chemical(dataall[dataall.place == 'CM'], chemical) for chemical in chemicals]
results_df_cm = pd.DataFrame(results_cm)
results_df_cm['Place'] = 'CM'  # 添加地点信息


In [ ]:
# 合并 JH 和 CM 的结果
results_df = pd.concat([results_df_jh, results_df_cm], axis=0)

results_df.set_index(['Chemical', 'Cluster', 'Place'], inplace=True)

jh_r2_mean = results_df.xs('JH', level='Place')['R2 Mean']

sorted_index = jh_r2_mean.sort_values(ascending=False).index.get_level_values('Chemical')
results_df_sorted = results_df.loc[sorted_index]

# 显示结果
results_df_sorted

In [ ]:
# 保存为 Excel 文件并指定 sheet 名称
results_df_sorted.to_excel("results_sci_format.xlsx", sheet_name="Results")
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side
# 打开保存的 Excel 文件，设置列宽和基本格式
with pd.ExcelWriter("results_sci_format.xlsx", engine="openpyxl") as writer:
    results_df_sorted.to_excel(writer, sheet_name="Results")
    workbook = writer.book
    worksheet = writer.sheets["Results"]

    # 设置字体为 Times New Roman 并调整列宽
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter  # 获取列字母
        for cell in col:
            cell.font = Font(name="Times New Roman", size=11)  # 设置字体和大小
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        # 根据内容调整列宽
        worksheet.column_dimensions[column].width = max(max_length + 2, 15)

    # 设置表头加粗
    for cell in worksheet[1]:
        cell.font = Font(name="Times New Roman", bold=True, size=11)